# Creating asset summaries

We'll need a *LINZ extension schema validator:*

In [1]:
from json import load
from sys import stderr
from urllib.request import urlopen

from jsonschema import Draft7Validator

with urlopen("https://stac.linz.govt.nz/v0.0.10/linz/schema.json") as schema_pointer:
    schema = load(schema_pointer)

validator = Draft7Validator(schema)

def validate(instance: str) -> None:
    for error in validator.iter_errors(instance):
        print(error.message, file=stderr)

The *collection example* is valid:

In [2]:
with urlopen("https://stac.linz.govt.nz/v0.0.10/linz/examples/collection.json") as example_pointer:
    example = load(example_pointer)

validate(example)

Can we *summarise assets?*

In [3]:
example["summaries"]["assets"] = {
    "created": {"minimum": "1999-01-01T00:00:00Z", "maximum": "2010-01-01T00:00:00Z"},
    "updated": {"minimum": "1999-01-02T00:00:00Z", "maximum": "2010-01-02T00:00:00Z"},
}
validate(example)

Yes, but *are such properties validated as normal summaries?* What if there's an invalid datetime or unexpected structure?

In [4]:
example["summaries"]["assets"] = {
    "created": None,
    "updated": {"minimum": "1999-01-02T00:00:00Z"},
}
validate(example)

Oops, that should've caused a *validation exception!* Let's instead extend the extension schema to validate asset summaries as normal summaries:

In [5]:
schema["definitions"]["fields"]["properties"]["summaries"]["properties"]["assets"] = {
    "$ref": "https://schemas.stacspec.org/v1.0.0/collection-spec/json-schema/collection.json#definitions/summaries"
}
validate(example)

None is not valid under any of the given schemas
{'minimum': 'not a datetime'} is not valid under any of the given schemas


Looks like that worked – the assets summary is validated like any other summary. Let's make sure by reverting to the valid summary:

In [6]:
example["summaries"]["assets"] = {
    "created": {"minimum": "1999-01-01T00:00:00Z", "maximum": "2010-01-01T00:00:00Z"},
    "updated": {"minimum": "1999-01-02T00:00:00Z", "maximum": "2010-01-02T00:00:00Z"},
}
validate(example)

And we're golden!